<a href="https://colab.research.google.com/github/mint-aguccim/Machine_Learning_Programming/blob/main/04_Main_Execution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
import numpy as np
from google.colab import drive

# 1. 드라이브 연결
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/P실무프로젝트_3조'
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
import numpy as np
from google.colab import drive

# 1. 드라이브 연결
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/P실무프로젝트_3조'

# 경로 설정
data_folder = os.path.join(base_path, 'data')
image_folder = os.path.join(base_path, 'images')
btc_image_folder = os.path.join(base_path, 'bitcoin_images')
model_folder = os.path.join(base_path, 'models')

# 2. 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 실행 장치: {device}")

# ==========================================
# [Model] 학습 때와 동일한 모델 구조 (5-Class)
# ==========================================
class HybridStockModel(nn.Module):
    def __init__(self):
        super(HybridStockModel, self).__init__()

        # CNN 파트
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten()
        )
        self.cnn_fc = nn.Linear(16 * 16 * 64, 128)

        # MLP 파트 (입력 3개: RSI, SMA, News)
        self.mlp = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU()
        )

        # 결합 파트
        self.final_fc = nn.Sequential(
            nn.Linear(128 + 16, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 5) # 1~5점 출력
        )

    def forward(self, image, features):
        x1 = self.cnn(image)
        x1 = self.cnn_fc(x1)
        x2 = self.mlp(features)
        combined = torch.cat((x1, x2), dim=1)
        output = self.final_fc(combined)
        return output

# 이미지 전처리 설정
img_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
# 경로 설정
data_folder = os.path.join(base_path, 'data')
image_folder = os.path.join(base_path, 'images')
btc_image_folder = os.path.join(base_path, 'bitcoin_images')
model_folder = os.path.join(base_path, 'models')

# 2. 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 실행 장치: {device}")

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 실행 장치: cpu
🚀 실행 장치: cpu


In [ ]:
# ==========================================
# [Report] 결과 리포트 생성기
# ==========================================
def generate_report(date, score, confidence, rsi, sma, news):
    print(f"\n📊 [{date}] AI 투자 분석 리포트")
    print("-" * 40)
    print(f"1. 입력 지표")
    print(f"   - 📈 기술적 점수: RSI({rsi}점) / SMA({sma}점)")
    print(f"   - 📰 뉴스 감성점수: {news}점")
    print("-" * 40)

    MIN_CONFIDENCE = 60.0  # # 진입 확신도. "60% 미만이면 진입 금지"

    # 전략 로직 (5점 만점 기준) # 투자 비중
    action = ""
    weight = 0
    msg = ""

    if score == 5:
        action = "🚀 강력 매수 (Strong Buy)"
        weight = 50 + (confidence - 50) # 확신도가 높을수록 더 많이
        msg = "급등 패턴이 감지되었습니다. 공격적으로 진입하세요."
    elif score == 4:
        action = "✅ 매수 (Buy)"
        weight = 30
        msg = "상승 추세입니다. 분할 매수로 접근하세요."
    elif score == 3:
        action = "👀 관망 (Hold)"
        weight = 0
        msg = "방향성이 불확실합니다. 현금을 보유하세요."
    elif score == 2:
        action = "⚠️ 매도 (Sell)"
        weight = -50
        msg = "하락 징후가 보입니다. 리스크 관리가 필요합니다."
    else: # score == 1
        action = "🚨 전량 매도 (Strong Sell)"
        weight = -100
        msg = "폭락 위험이 큽니다. 즉시 탈출하세요."

    # 비중 상한선
    if weight > 100: weight = 100
    if weight < -100: weight = -100

    print(f"2. AI 예측 결과")
    print(f"   - 🏆 종합 등급: {score}점 / 5점")
    print(f"   - 🤖 AI 확신도: {confidence:.2f}%")
    print("-" * 40)
    print(f"📢 최종 판단: {action}")

    if weight > 0:
        print(f"💰 추천 비중: 현금의 {weight:.1f}% 투입")
    elif weight == 0:
        print(f"💰 추천 행동: 매매 쉬기")
    else:
        print(f"💰 추천 행동: 보유 주식 {abs(weight)}% 매도")

    print(f"💬 한줄평: {msg}")
    print("=" * 40)

# ==========================================
# [Predict] 예측 실행 함수
# ==========================================
def predict_today(model_name, date, img_path, rsi_score, sma_score, news_score):

    # 1. 모델 불러오기
    model_path = os.path.join(model_folder, model_name)
    model = HybridStockModel().to(device)

    if not os.path.exists(model_path):
        print(f"❌ 모델 파일이 없습니다: {model_name}")
        return

    try:
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval()
    except Exception as e:
        print(f"❌ 모델 로드 중 에러 발생: {e}")
        return

    # 2. 이미지 준비
    if not os.path.exists(img_path):
        print(f"❌ 이미지 파일이 없습니다: {img_path}")
        return

    try:
        image = Image.open(img_path).convert('RGB')
        image = img_transforms(image).unsqueeze(0).to(device)
    except:
        print("❌ 이미지 처리 중 에러 발생")
        return

    # 3. 숫자 데이터 준비
    features = torch.tensor([[rsi_score, sma_score, news_score]], dtype=torch.float32).to(device)

    # 4. AI 추론
    with torch.no_grad():
        outputs = model(image, features)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)

        top_prob, top_class = torch.max(probabilities, 1)

        # 0~4 인덱스를 1~5점으로 변환
        predicted_score = top_class.item() + 1
        confidence = top_prob.item() * 100
    # 5. 커스텀 가중치 로직 적용
    # 5-1. 가중치 설정 (합이 1.0이 되도록 설정해보세요)
    # 예시: 뉴스(Ear)에 50% 힘을 실어주고 싶을 때
    w_eye = 0.3    # 차트(AI) 비중  # 가중치
    w_ear = 0.5    # 뉴스 비중
    w_trend = 0.2  # 보조지표(RSI+SMA) 비중

    # 5-2. 보조지표 평균 계산 (RSI와 SMA의 평균)
    trend_avg = (rsi_score + sma_score) / 2

    # 5-3. 가중 합산 점수 계산
    final_weighted_score = (predicted_score * w_eye) + (news_score * w_ear) + (trend_avg * w_trend)

    # 5-4. 반올림해서 최종 1~5점 정수로 만들기
    final_score = int(round(final_weighted_score))

    # (혹시 계산 결과가 1보다 작거나 5보다 크면 보정)
    if final_score < 1: final_score = 1
    if final_score > 5: final_score = 5

    # (디버깅용 출력) 가중치 적용 전후 비교
    print(f"   👉 [가중치 적용] AI원래점수({predicted_score}) + 뉴스({news_score}) + 추세({trend_avg}) => 최종({final_score}점)")

    # 6. 리포트 출력
    generate_report(date, final_score, confidence, rsi_score, sma_score, news_score)

In [ ]:
# ==========================================
# [Main] 실전 예측 가동
# ==========================================

# 데이터셋에서 가장 최근 데이터 1줄 가져오기
def get_latest_data(csv_name):
    path = os.path.join(data_folder, csv_name)
    if not os.path.exists(path):
        print(f"⚠️ 데이터 파일이 없습니다: {csv_name}")
        return None
    df = pd.read_csv(path)
    return df.iloc[-1] # 마지막 행

# --- 실행 1: 삼성전자 ---
print("\n🔵 [삼성전자] 예측 시스템 가동...")
samsung_data = get_latest_data('final_dataset_stock.csv')

if samsung_data is not None:
    # 이미지 경로 조립
    img_full_path = os.path.join(image_folder, samsung_data['Image_Path'])

    predict_today(
        model_name='samsung_model.pth',
        date=samsung_data['Date'],
        img_path=img_full_path,
        rsi_score=samsung_data['RSI_Score'],
        sma_score=samsung_data['SMA_Score'],
        news_score=samsung_data['News_Score']
    )

# --- 실행 2: 비트코인 ---
print("\n🟠 [비트코인] 예측 시스템 가동...")
btc_data = get_latest_data('final_dataset_coin.csv')

if btc_data is not None:
    img_full_path_btc = os.path.join(btc_image_folder, btc_data['Image_Path'])

    predict_today(
        model_name='bitcoin_model.pth',
        date=btc_data['Date'],
        img_path=img_full_path_btc,
        rsi_score=btc_data['RSI_Score'],
        sma_score=btc_data['SMA_Score'],
        news_score=btc_data['News_Score']
    )


🔵 [삼성전자] 예측 시스템 가동...
   👉 [가중치 적용] AI원래점수(3) + 뉴스(3) + 추세(2.5) => 최종(3점)

📊 [2024-05-17] AI 투자 분석 리포트
----------------------------------------
1. 입력 지표
   - 📈 기술적 점수: RSI(3점) / SMA(2점)
   - 📰 뉴스 감성점수: 3점
----------------------------------------
2. AI 예측 결과
   - 🏆 종합 등급: 3점 / 5점
   - 🤖 AI 확신도: 26.17%
----------------------------------------
📢 최종 판단: 👀 관망 (Hold)
💰 추천 행동: 매매 쉬기
💬 한줄평: 방향성이 불확실합니다. 현금을 보유하세요.

🟠 [비트코인] 예측 시스템 가동...
   👉 [가중치 적용] AI원래점수(3) + 뉴스(3.0) + 추세(4.0) => 최종(3점)

📊 [2024-05-18] AI 투자 분석 리포트
----------------------------------------
1. 입력 지표
   - 📈 기술적 점수: RSI(3점) / SMA(5점)
   - 📰 뉴스 감성점수: 3.0점
----------------------------------------
2. AI 예측 결과
   - 🏆 종합 등급: 3점 / 5점
   - 🤖 AI 확신도: 23.69%
----------------------------------------
📢 최종 판단: 👀 관망 (Hold)
💰 추천 행동: 매매 쉬기
💬 한줄평: 방향성이 불확실합니다. 현금을 보유하세요.
